SOLUCIÓN TALLER #2

1. Considere una red neural con 2 variables de entrada y una sola capa oculta con 2 neuronas, que se conectan a una única neurona de salida, es decir 5 neuronas en total. Asuma que para todas las neuronas se utiliza el sigmoide como función de activación. ¿Cuántos parámetros entrenables o pesos tendría este modelo sencillo? Como función objetivo o de pérdidas, se utiliza el error cuadrático promedio (Mean squared error), asuma que el bias de todas las neuronas es cero. Describa cómo podría implementarse el algoritmo de backpropagation para entrenar esta red. Encuentre expresiones analíticas para el gradiente de la función de pérdidas con respecto a todos los parámetros entrenables del modelo.

Solución: 

1. En ese caso tenemos 2 variables de entrada, 2 neuronas en la capa oculta, 1 neurona de salida, en total 5 neuronas. 
Para las conexiones entre la capa de entrada y la capa oculta habrán 2 pesos por capa, es decir 4 pesos en total, además por cada neuoran en la capa oculta habrá un sesgo bias, serán 2 pero en 
este caso serán 0. 

Para las conexiones entre la capa oculta y la capa de salida habrán dos neuronas * 1 peso por cada una = 2 pesos en total. La neurona de salida tendrá un sesgo también. 

Entonces, el número total de parámetros entrenables será: 

a. Pesos entre la capa de entrada y la capa oculta: 4
b. Pesos entre la capa oculta y la capa de salida: 2

Entonces tenemos 6 parámetros entrenables teniendo en cuenta que los sesgos biasson 0. 

2. Como podría implementarse el algoritmo de backpropagation:


In [ ]:
import numpy as np

# 1er paso, definimos la función sigmoide 
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Gradiente de la función de pérdida con respecto a v_j
def grad_loss_wrt_output_weights(y, y_target, h_j):
    return (y - y_target) * y * (1 - y) * h_j

# Gradiente de la función de pérdida con respecto a w_ij
def grad_loss_wrt_hidden_weights(y, y_target, v_j, h_j, x_i):
    return (y - y_target) * y * (1 - y) * v_j * h_j * (1 - h_j) * x_i

# Voy a tomar unos datos de ejemplo para una prueba 
def main():
    # Datos de entrada
    x1 = 0.5
    x2 = 0.3
    
    # Pesos
    w11 = 0.4
    w12 = -0.2
    w21 = 0.1
    w22 = 0.8
    v1 = -0.5
    v2 = 0.7
    
    # Calculamos las salidas de la capa oculta
    h1 = sigmoid(w11 * x1 + w21 * x2)
    h2 = sigmoid(w12 * x1 + w22 * x2)
    
    # Calculamos la salida de la capa de salida
    y = sigmoid(v1 * h1 + v2 * h2)
    
    # Target
    y_target = 0.8
    
    # Gradientes
    grad_v1 = grad_loss_wrt_output_weights(y, y_target, h1)
    grad_v2 = grad_loss_wrt_output_weights(y, y_target, h2)
    grad_w11 = grad_loss_wrt_hidden_weights(y, y_target, v1, h1, x1)
    grad_w12 = grad_loss_wrt_hidden_weights(y, y_target, v2, h2, x1)
    grad_w21 = grad_loss_wrt_hidden_weights(y, y_target, v1, h1, x2)
    grad_w22 = grad_loss_wrt_hidden_weights(y, y_target, v2, h2, x2)
    
    print("Gradiente de pérdida con respecto a v1:", grad_v1)
    print("Gradiente de pérdida con respecto a v2:", grad_v2)
    print("Gradiente de pérdida con respecto a w11:", grad_w11)
    print("Gradiente de pérdida con respecto a w12:", grad_w12)
    print("Gradiente de pérdida con respecto a w21:", grad_w21)
    print("Gradiente de pérdida con respecto a w22:", grad_w22)

if __name__ == "__main__":
    main()


Punto 1.2

In [ ]:
conda install torch pandas

In [3]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [6]:
#Leer datos csv

data = pd.read_csv("Datos GHF\global_ghf.csv")
print(data.shape)


(4034, 21)


In [9]:
#Obtener valores x e y para la predicción

x = data.iloc[:, np.r_[0:14, 16:21]].values  # Selecciona todas las columnas excepto la columna 15 (que es donde está el valor de GHF)
y = data.iloc[:, 15].values #Seleccionamos la columna 15 exacta

In [10]:
# Dividir los datos en datos de entrenamiento y datos de prueba 

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [11]:
# Escalamos los datos

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
# Convertimos los datos a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [13]:
# Creamos un DataLoader para facilitar el manejo de los datos durante el entrenamiento, basicamente los vamos a cargar 
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) #El bach lo podemos ir cambiando según como veamos el rendimiento del modelo 

In [41]:
# Definimos una clase que será la arquitectura de la red neuronal
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)  # Capa completamente conectada
        self.fc2 = nn.Linear(32, 16)          # Capa completamente conectada
        self.fc3 = nn.Linear(16, 1)           # Capa de salida

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Función de activación ReLU en la primera capa oculta, en este caso vamos a usar ReLU pero podemos ubicar otras funciones
                                     # de activación si obtenemos resultados no tan favorables
        x = torch.relu(self.fc2(x))  # Función de activación ReLU en la segunda capa oculta
        x = self.fc3(x)              # Sin función de activación en la capa de salida
        return x

In [42]:
# Instanciar el modelo
input_size = X_train.shape[1]
model = NeuralNetwork(input_size)

In [43]:
# Definimos la función de pérdida y el optimizador
criterion = nn.MSELoss()                   # Mean Squared Error Loss, es la que usamos en el taller 1
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Optimizador Adam (esto es generalmente usado, lo tomé de ejemplo pero hay muchos optimizadores, no he probado muchos)

In [44]:
# Entrenamiento del modelo
num_epochs = 10 #Lo vamos modificando en función de la calidad de los resultados
for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()            # Reiniciar los gradientes acumulados (es importante para las iteraciones)
        outputs = model(inputs)         # Forward pass
        loss = criterion(outputs, targets.unsqueeze(1))  # Calcular la pérdida (iremos ajustando las pérdidas para ir mejorando el modelo)
        loss.backward()                 # Backward pass
        optimizer.step()                # Actualizar los parámetros
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 2.6621
Epoch [2/10], Loss: 0.4253
Epoch [3/10], Loss: 0.1682
Epoch [4/10], Loss: 0.1609
Epoch [5/10], Loss: 0.1885
Epoch [6/10], Loss: 0.1265
Epoch [7/10], Loss: 0.1759
Epoch [8/10], Loss: 0.0771
Epoch [9/10], Loss: 0.0883
Epoch [10/10], Loss: 0.1058


In [45]:
#Probamos el modelo para ver su calidad 

with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_pred, y_test_tensor.unsqueeze(1))
    print(f"Test Loss: {test_loss.item():.4f}")

#En esta prueba, obtuvimos una pérdida en nuestro test de 0.06, bastante buena, básicamente podemos decir que el modelo tiene un excelente nivel de predicción

Test Loss: 0.0901


In [46]:
# Guardar el modelo 

torch.save(model.state_dict(), "modelo_entrenado.pth")


In [48]:
model = NeuralNetwork(input_size)  
model.load_state_dict(torch.load("modelo_entrenado.pth"))
model.eval()  # Modelo en modo de evaluación

NeuralNetwork(
  (fc1): Linear(in_features=19, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=1, bias=True)
)

In [ ]:
#Definimos unos datos de ejemplo
data_test = [0,12.999,13,0.060800001,2048.46,2135.04,3879.25,1764.58,30.50409809,1.980000019,36.3899994,622.95,-110.333313,75.50012207,2.29706001,-1.69836,-42.0954018,219571,110.1439972,76.2264023]
# El modelo debe predecir GHF de 54

data_tensor = torch.tensor(data_test, dtype=torch.float32)

new_data_tensor = data_tensor.view(20, 1)

with torch.no_grad():
    predictions = model(new_data_tensor)

print("Predicciones:")
print(predictions)
